# Your first Trove API request

<p class="alert alert-info">New to Jupyter notebooks? Try <a href="getting-started/Using_Jupyter_notebooks.ipynb"><b>Using Jupyter notebooks</b></a> for a quick introduction.</p>

In this section we're going to learn how to send a request for information to the Trove API.

API requests are just like normal urls. However, instead of sending us back a web page, they deliver data in a form that computers can understand. We can then use that data in our own programs.

We're going to use the Python [Requests](http://docs.python-requests.org/en/master/) module to handle our API queries, so let's import it now.

In [27]:
import os

# We're going to use the Python Requests module to handle out API queries
import requests

# We'll use this to display nice;y formatted JSON results
from IPython.display import JSON

In [18]:
%%capture
# Load variables from the .env file if it exists
# Use %%capture to suppress messages
%load_ext dotenv
%dotenv

## Getting an API key
Any requests you make to the Trove API need to be authenticated with a 'key'. For non-commercial projects, you just  fill out a simple form and your API key is generated instantly. Follow the instructions in the Trove Help to [obtain your own Trove API Key](http://help.nla.gov.au/trove/building-with-trove/api).

Once you've created a key, you can access it at any time on the 'For developers' tab of your Trove user profile.

Copy your API key now, and paste it in the cell below, between the quotes.

In [ ]:
# This creates a variable called 'api_key', paste your key between the quotes
api_key = ""

# Use an api key value from environment variables if it is available (useful for testing)
if os.getenv("TROVE_API_KEY"):
    api_key = os.getenv("TROVE_API_KEY")

# This displays a message with your key
print("Your API key is: {}".format(api_key))

## Preparing parameters

All search queries to the Trove API start with the same base url. We'll save it as a variable here.

In [20]:
# Create a variable called 'api_search_url' and give it a value
api_search_url = "https://api.trove.nla.gov.au/v2/result"

Trove API queries are constructed by adding parameters to the base url. Most of the parameters are optional, but a few are mandatory:

* `q` &ndash; 'q' for query, this is where search terms go
* `zone` &ndash; which Trove zone (or zones) do you want to search, use 'all' for everything
* `key` &ndash; your Trove API key

If you don't want to specify a search term, you can just use a space or a plus sign &ndash; '&nbsp;' or '+' &ndash; as the value for `q`. Of course, this means that you're asking for *everything*, so Trove might take a bit longer to respond.

We'll meet some other parameters later, but for now let's create a Python dictionary to store our basic parameters. The `requests` library will take this dictionary, turn it into a string, and add it to the base url.

For our first API request we're going to search Trove's digitised newspapers, so we'll assign the value 'newspaper' to the `zone` parameter. Feel free to edit the `q` value to search for something that interests you.


In [21]:
# This creates a dictionary called 'params' and sets values for the API's mandatory parameters
params = {
    "q": "cyclone",  # Search for this keyword -- feel free to change!
    "zone": "newspaper",  # Search in the newspaper zone
    "key": api_key,
}

The default output of the API is XML. For most applications it's easier to work with JSON. You set this using the `encoding` parameter. Let's add this into `params` and view the result.

In [22]:
# This adds a value for 'encoding' to our dictionary
params["encoding"] = "json"

# Let's view the updated dictionary
params

{'q': 'cyclone',
 'zone': 'newspaper',
 'key': 'gq29l1g1h75pimh4',
 'encoding': 'json'}

## Making a request

Ok, we're now now ready to make our first query!

In [23]:
# This sends our request to the Trove API and stores the result in a variable called 'response'
response = requests.get(api_search_url, params=params)

# This shows us the url that's sent to the API
print(
    "Here's the formatted url that gets sent to the Trove API:\n{}\n".format(
        response.url
    )
)

# This checks the status code of the response to make sure there were no errors
if response.status_code == requests.codes.ok:
    print("All ok")
elif response.status_code == 403:
    print("There was an authentication error. Did you paste your API key above?")
else:
    print("There was a problem. Error code: {}".format(response.status_code))
    print("Try running this cell again.")

Here's the formatted url that gets sent to the Trove API:
https://api.trove.nla.gov.au/v2/result?q=cyclone&zone=newspaper&key=gq29l1g1h75pimh4&encoding=json

All ok


See how `requests` has taken our parameters and turned them into a string with '&' between each one? 

The url above is live &ndash; try clicking on it to see the raw results from Trove.

## Look at the results

The `response` variable contains all the data returned to us by the Trove API. Let's get it out in a usable form.

In [24]:
# Get the Trove API's JSON results and make them available as a Python variable called 'data'
data = response.json()

In [31]:
# Display the results as nicely-formatted JSON
JSON(data, expanded=True)

<IPython.core.display.JSON object>

## Do something with the data

As you can see, the API results are fairly complex. Individual item records are quite deeply nested. In a future section we'll explore this structure in more detail. But for now, let's run a simple script to display the basic details of each of our matching articles.

In [29]:
# Loop through all the newspaper articles
# The articles themselves are quite deeply nested, so we have to go down several levels to get them
for article in data["response"]["zone"][0]["records"]["article"]:

    # Display a string containing the date, title, newspaper, and page for each article
    print(
        f'{article["date"]}, "{article["heading"]}", {article["title"]["value"]}, page {article["page"]}'
    )

1872-03-23, "CYCLONE.", The Herald (Fremantle, WA : 1867 - 1886), page 3
1954-03-11, "CYCLONE", Catholic Weekly (Sydney, NSW : 1942 - 1954), page 18
1976-02-18, "Cyclone", The Canberra Times (ACT : 1926 - 1995), page 16
1949-01-20, "CYCLONE", The Grenfell Record and Lachlan District Advertiser (NSW : 1876 - 1951), page 3
1906-11-28, "Cyclone.", The Armidale Chronicle (NSW : 1894 - 1929), page 4
1892-02-26, "CYCLONE.", The Australian Advertiser (Albany, WA : 1888 - 1897), page 3
1882-02-08, "Cyclone.", Mackay Mercury and South Kennedy Advertiser (Qld. : 1867 - 1887), page 2
1996-10-18, "Cyclone", The Australian Jewish News (Melbourne, Vic. : 1935 - 1999), page 4
1977-01-12, "Cyclone", The Canberra Times (ACT : 1926 - 1995), page 3
1893-09-01, "CYCLONE.", Zeehan and Dundas Herald (Tas. : 1890 - 1922), page 2
1969-02-11, "CYCLONE", The Canberra Times (ACT : 1926 - 1995), page 6
1904-03-23, "CYCLONE.", Cootamundra Herald (NSW : 1877 - 1954), page 2
1972-04-13, "Cyclone", The Canberra Times

## Congratulations!

You've made your first Trove API request. Now let's move on to learn a bit [about Trove's zones](Working-with-zones.ipynb).

----

Created by [Tim Sherrratt](https://timsherratt.org) for the [GLAM workbench](https://glam-workbench.net/). Support this project by [becoming a GitHub sponsor](https://github.com/sponsors/wragge?o=esb).